In [1]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import sort
from sklearn.feature_selection import SelectFromModel

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from itertools import combinations

In [14]:
df = pd.read_csv("TE-Train.csv")
#df = df.drop(["year","name","college","position"],1)
df.head()
# 1009

,year,name,college,position,height_in,weight_lbs,40yards,benchpress,vert_leap_in,broad_jump_in,...,allamerican,hasawards,heisman,hasbling,scrimyds,scrimplays,scrimypa,scrimtds,hasstats,drafted
0,1997,john allred,Southern California,TE,76.4,244,5.01,15.0,32.0,112.0,...,0.0,0.0,0.0,0.0,589.0,54.0,10.9,3.0,NaN,Y
1,1997,pete chryplewicz,Notre Dame,TE,76.6,253,4.88,NaN,30.5,111.0,...,0.0,0.0,0.0,0.0,585.0,48.0,12.2,5.0,NaN,Y
2,1997,greg clark,Stanford,TE,76.4,262,4.99,30.0,31.0,115.0,...,0.0,0.0,0.0,0.0,492.0,43.0,11.4,3.0,NaN,Y
3,1997,pat fitzgerald,Texas,TE,74.1,237,5.00,19.0,29.5,107.0,...,0.0,0.0,0.0,0.0,1109.0,80.0,13.9,12.0,NaN,Y
4,1997,tony gonzalez,California,TE,76.1,244,4.83,NaN,33.5,116.0,...,1.0,1.0,0.0,1.0,1302.0,89.0,14.6,8.0,NaN,Y


In [3]:
from splinter import Browser
from sys import platform
import os
import csv
import re

executable_path = None

if platform == "linux" or platform == "linux2":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-linux")}
elif platform == "darwin":
    executable_path = {'executable_path': os.path.join("binaries", "chromedriver-mac")}
elif platform == "win32":
    executable_path = {'executable_path': os.path.join("..","..","binaries", "chromedriver-windows.exe")}

browser = Browser('chrome', **executable_path, headless=False)

In [15]:
for index, row in df.iterrows():
    if (df.loc[index,"hasbling"] != 1.0):
        if (df.loc[index,"hasbling"] != 0.0):
            x = row["name"]
            browser.visit(f"https://www.sports-reference.com/cfb/search/search.fcgi?hint=&search={x}")
            if(len(browser.find_by_css("h1[itemprop='name']")) == 1):
                try:
                    df.loc[index, "scrimyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                    df.loc[index, "scrimplays"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_att']").text
                    df.loc[index, "scrimypa"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds_per_att']").text
                    df.loc[index, "scrimtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                except:
                    df.loc[index, "scrimyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                    df.loc[index, "scrimplays"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_att']").text
                    df.loc[index, "scrimypa"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds_per_att']").text
                    df.loc[index, "scrimtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                            
                if (len(browser.find_by_css("ul#bling")) > 0):
                    df.loc[index,"hasbling"] = 1
                else:
                    df.loc[index,"hasbling"] = 0
                if (len(browser.find_by_css("li.bling_all_star")) > 0):
                    df.loc[index,"allamerican"] = 1
                else:
                    df.loc[index,"allamerican"] = 0
                if (len(browser.find_by_css("li.bling_hof")) > 0):
                    df.loc[index,"heisman"] = 1
                else:
                    df.loc[index,"heisman"] = 0
                if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                    df.loc[index,"hasawards"] = 1
                else:
                    df.loc[index,"hasawards"] = 0
            elif (len(browser.find_by_css("div.search-item")) == 0):
                df.loc[index,"hasbling"] = 0
                df.loc[index,"allamerican"] = 0
                df.loc[index,"heisman"] = 0
                df.loc[index,"hasawards"] = 0
            else:
                listed = browser.find_by_css("div.search-item")

                for x in listed:
                    if (re.sub(r'\(.*\)', "", row["college"]).strip() in x.find_by_css("div.search-item-league").text):
                        x.find_by_css("div.search-item-name a").click()
                        if (len(browser.find_by_css("h1[itemprop='name']")) == 1):
                            try:
                                df.loc[index, "scrimyds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                                df.loc[index, "scrimplays"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_att']").text
                                df.loc[index, "scrimypa"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds_per_att']").text
                                df.loc[index, "scrimtds"] = browser.find_by_css("table#receiving tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                            except:
                                df.loc[index, "scrimyds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds']").text
                                df.loc[index, "scrimplays"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_att']").text
                                df.loc[index, "scrimypa"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_yds_per_att']").text
                                df.loc[index, "scrimtds"] = browser.find_by_css("table#rushing tfoot tr.bold")[0].find_by_css("td[data-stat='scrim_td']").text
                            
                            if (len(browser.find_by_css("ul#bling")) > 0):
                                df.loc[index,"hasbling"] = 1
                            else:
                                df.loc[index,"hasbling"] = 0
                            if (len(browser.find_by_css("li.bling_all_star")) > 0):
                                df.loc[index,"allamerican"] = 1
                            else:
                                df.loc[index,"allamerican"] = 0
                            if (len(browser.find_by_css("li.bling_hof")) > 0):
                                df.loc[index,"heisman"] = 1
                            else:
                                df.loc[index,"heisman"] = 0
                            if (len(browser.find_by_css("div#leaderboard_awards")) > 0):
                                df.loc[index,"hasawards"] = 1
                            else:
                                df.loc[index,"hasawards"] = 0
                            break
                        else:
                            df.loc[index,"hasbling"] = 0
                            df.loc[index,"allamerican"] = 0
                            df.loc[index,"heisman"] = 0
                            df.loc[index,"hasawards"] = 0
                            break
                    else:
                        df.loc[index,"hasbling"] = 0
                        df.loc[index,"allamerican"] = 0
                        df.loc[index,"heisman"] = 0
                        df.loc[index,"hasawards"] = 0

In [17]:
df.to_csv("TE-train.csv", index=False)